# Практическая работа
# Компьютерное зрение
# Обучение с передачей знаний для распознавания повреждений глазной сетчатки
## Преамбула
В этой практической работе вы попробуете построить модель глубокой нейронной сети для определения класса повреждения глазной сетчатки по снимкам ОКТ.

Однако, глубокие нейронные сети требовательны к большим объемам данных для сходимости обучения. И зачастую в нашей частной задаче недостаточно данных для того, чтобы хорошо натренировать все слои нейросети. `Transfer Learning` (`обучение переносом`) решает эту проблему.

`Transfer Learning` (`трансферное обучение`) — это подраздел машинного обучения, целью которого является применение знаний, полученные из одной задачи, к другой целевой задаче.

Для таких типовых задач, как классификация изображений, можно воспользоваться готовой архитектурой (AlexNet, VGG, Inception, ResNet и т.д.) и обучить нейросеть на своих данных. Реализации таких сетей с помощью различных фреймворков уже существуют, так что в этой работе мы будем использовать одну из них как черный ящик, не вникая глубоко в принцип её работы.

Нейронные сети, которые используются для классификации, как правило, содержат $N$ выходных нейронов в последнем слое, где $N$ — это количество классов. Такой выходной вектор трактуется как набор вероятностей принадлежности к классу. В нашей задаче количество классов может отличаться от того, которое было в исходном датасете. В таком случае нам придётся полностью выкинуть этот последний слой и поставить новый, с нужным количеством выходных нейронов.

![Текст ссылки](https://habrastorage.org/r/w1560/webt/u_/n3/k3/u_n3k3qpkps6nw9tjjzwc0njl-y.jpeg)

[Вспомогательные материалы](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)

## Данные

`Оптическая когерентная томография сетчатки (ОКТ)` $-$ это метод визуализации, используемый для получения поперечных срезов сетчатки живых пациентов с высоким разрешением. Ежегодно выполняется около 30 миллионов ОКТ-сканирований, и анализ и интерпретация этих изображений занимает значительное количество времени (Swanson and Fujimoto, 2017).

![Текст ссылки](https://i.imgur.com/fSTeZMd.png)

На рисунке представлены классы повреждений сетчатки:

* (Крайний слева) неоваскуляризация хориоидеи (CNV) с неоваскулярной мембраной (белые стрелки) и связанной с ней субретинальной жидкостью (стрелки);
* (Средний слева) Диабетический макулярный отек (ДМЭ) с интраретинальной жидкостью, связанной с утолщением сетчатки (стрелки);
* (Справа посередине) Множественные друзы (наконечники стрел) присутствуют на ранних стадиях ВМД;
*(Крайний справа) Нормальная сетчатка с сохраненным контуром ямки и отсутствием какой-либо жидкости / отека сетчатки.

В этой задаче вы будете работать действительно с большим набором данных. Код для скачивания данных представлен ниже. Скачивание занимает в среднем 5 минут.

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("paultimothymooney/kermany2018")

print("Path to dataset files:", path)

In [ ]:
import os # модуль для работы с операционной системой
os.listdir(path) # возвращает список вложенных в директорию с указанным путем объектов

Выберите директорию `OCT2017`. Лежащие в ней данные используйте для выполнения заданий.
Набор данных уже разделён на три выборки: `train`, `val` и `test`. Первые два поднабора будут использоваться для обучения, один $-$ для тестирования.

In [ ]:
# скажите, сколько файлов в каждом поднаборе
# ваш код здесь

## Построение модели

In [ ]:
import torch
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt

# выберите устройство, на котором будете обучать нейронную сеть

В этой задаче загрузить данные достаточно просто, если использовать класс `torchvision.datasets.ImageFolder`.

Что нужно сделать:
* Задайте необходимую цепочку преобразований для картинок, используя `torchvision.transofoms`. Загляните в документацию к этому модулю и найдите преобразования: случайная обрезка (обрежьте случайным образом картинки до размера 224x224), случайный поворот, превратите в `torch.Tensor`, как делали это в предыдущей работе;

* Соберите три датасета с помощью `torchvision.datasets.ImageFolder`;

* Создайте три загрузчика данных (`torch.utils.data.DataLoader`). Для каждого из них укажите размер батча, укажите параметр, отвечающий за перетасовку изображений внутри каждой папки (нам не важен порядок картинок, поэтому случайная перетасовка уберёт лишние зависимости, которым может обучиться модель).

In [ ]:
# ваш код здесь

Покажите, какие изображения присутствуют в выборке. Напишите функцию `imshow`, выводящую на экран изображения в одном батче.

In [ ]:
def imshow(inp, title=None):
    # ваш код здесь
    pass

def show_databatch(inputs, classes):
    out = torchvision.utils.make_grid(inputs)
    imshow(out, title=[class_names[x] for x in classes])

# next позволяет сделать один шаг вперёд по загрузчику данных
inputs, classes = next(iter(dataloader_train))
show_databatch(inputs, classes)

Предобученная модель `VGG-16` способна классифицировать 1000 различных меток; нам просто нужно 4 вместо них. Чтобы сделать это, мы собираемся заменить последний полносвязный уровень модели на новый с 4 выходными нейронами вместо 1000.

В `PyTorch` мы можем получить доступ к классификатору `VGG-16` с помощью модуля `model.classifier`, который представляет собой 6-слойный массив. Мы заменим последнюю запись.

Скачайте и загрузите веса модели: [веса модели на kaggle](https://www.kaggle.com/datasets/pytorch/vgg16bn)

In [ ]:
from torchvision import models

vgg16 = models.vgg16_bn()
vgg16.load_state_dict(torch.load("путь до файла .pth"))
print(vgg16.classifier[6].out_features) # 1000

In [ ]:
# заморозьте все уровни модели vgg16
# ваш код

# затем добавьте новый слой
num_features = # количество входных нейронов для 6-го слоя
features = list(vgg16.classifier.children())[:-1] # удаляем последний (выходной) слой
features.extend() # добавьте полносвязный слой с 4-мя нейронами
vgg16.classifier = nn.Sequential(*features) # замените модель на новую в атрибуте classifier
print(vgg16)

Выберите оптимизатор обучения модели и функцию потерь.

Саму модель перенесите на `gpu`.

In [ ]:
# ваш код здесь

Напишите функции `train_model` для обучения и `eval_model` для оценки модели. Метрикой выберите `accuracy`. Предусмотрите, что эта модель может очень долго обучаться. Попробуйте предусмотреть сохранение весов очередной эпохи обучения и загрузку их в зависимости от какого-нибудь параметра параметра функции.

In [ ]:
def eval_model(vgg, criterion):
  pass

def train_model(vgg, criterion, optimizer, scheduler, num_epochs=10):
  pass

В конце оцените `accuracy` полученной модели.

In [ ]:
# ваш код